In [2]:
import tensorflow as tf
import keras
from keras import layers

inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)


Fit model on training data
Epoch 1/2
782/782 [==============================] - 7s 6ms/step - loss: 0.3423 - sparse_categorical_accuracy: 0.9012 - val_loss: 0.1757 - val_sparse_categorical_accuracy: 0.9492
Epoch 2/2
782/782 [==============================] - 5s 6ms/step - loss: 0.1594 - sparse_categorical_accuracy: 0.9519 - val_loss: 0.1395 - val_sparse_categorical_accuracy: 0.9599
Evaluate on test data
79/79 [==============================] - 0s 2ms/step - loss: 0.1364 - sparse_categorical_accuracy: 0.9576
test loss, test acc: [0.13639435172080994, 0.9575999975204468]
Generate predictions for 3 samples
1/1 [==============================] - 0s 45ms/step
predictions shape: (3, 10)


In [5]:
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    return model

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(64)

model.fit(train_dataset, epochs=3)

print("Evaluate")
result = model.evaluate(test_dataset)

dict(zip(model.metrics_names, result))


Epoch 1/3
782/782 [==============================] - 5s 6ms/step - loss: 0.1144 - sparse_categorical_accuracy: 0.9659
Epoch 2/3
782/782 [==============================] - 5s 6ms/step - loss: 0.0909 - sparse_categorical_accuracy: 0.9726
Epoch 3/3
782/782 [==============================] - 5s 6ms/step - loss: 0.0757 - sparse_categorical_accuracy: 0.9772
Evaluate
157/157 [==============================] - 0s 3ms/step - loss: 0.1051 - sparse_categorical_accuracy: 0.9693


{'loss': 0.10512403398752213,
 'sparse_categorical_accuracy': 0.9692999720573425}

In [6]:
model = get_compiled_model()

# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Prepare the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

model.fit(train_dataset, epochs=1, validation_data=val_dataset)

782/782 [==============================] - 6s 7ms/step - loss: 0.3328 - sparse_categorical_accuracy: 0.9055 - val_loss: 0.1982 - val_sparse_categorical_accuracy: 0.9412
